In [1]:
import pandas as pd
import numpy as np

##### Q1
load the ”Sleep Efficiency.csv” file into Pandas

In [2]:
sleep = pd.read_csv('./Sleep_Efficiency.csv')

##### Q2 
how many different entries are there for each columns.

In [3]:
sleep.describe().loc['count']

ID                        452.0
Age                       452.0
Sleep duration            452.0
Sleep efficiency          452.0
REM sleep percentage      452.0
Deep sleep percentage     452.0
Light sleep percentage    452.0
Awakenings                432.0
Caffeine consumption      427.0
Alcohol consumption       436.0
Exercise frequency        446.0
Name: count, dtype: float64

All columns have 452 observations except for awakening (432), Caffeine Cons. (427), Alcohol Cons. (436), and Exercise Freq. (446)

##### Q3 
how many entries are missing for each column

In [4]:
452 - sleep.describe().loc['count']

ID                         0.0
Age                        0.0
Sleep duration             0.0
Sleep efficiency           0.0
REM sleep percentage       0.0
Deep sleep percentage      0.0
Light sleep percentage     0.0
Awakenings                20.0
Caffeine consumption      25.0
Alcohol consumption       16.0
Exercise frequency         6.0
Name: count, dtype: float64

In [5]:
sum(sleep.isna().sum()) # There are 67 individual sleeppoints that are missing

67

In [6]:
sleep.isna().sum(axis=1).value_counts() # one person is missing two sleeppoints

0    386
1     65
2      1
Name: count, dtype: int64

In [7]:
sum(sleep.isna().sum(axis=1)>0) # 66 Individuals have missing sleep

66

In [8]:
missing_features = sleep.loc[:,sleep.isna().sum()!=0].columns.values
missing_features

array(['Awakenings', 'Caffeine consumption', 'Alcohol consumption',
       'Exercise frequency'], dtype=object)

Awakenings is missing 20 observations, Caffeine 25, Alcohol 16, and Exercise 6.  All other columns have no missing values.

##### Q4
for each missing entry, impute the missing values using the
following procedure:\
a) find 5 peoples with the same gender with the closest age
(”neighbors”)\
b) compute the average of these ”neighbors” as the value

Steps: 
1) find columns with missing values.
2) get indices of missing values and for each missing value and store missing column indices:
    a) get 5 closest same-sex neighbors by age and store their info for missing values.
    b) average and impute

Outer loop: missing features
inner loop (per feature): missing observations

def rand_select_cdf(inx_vals):
    """take in a list of (index, value) tuples and return a list of the indices
    corresponding to the smallest 5 values, where any ties are randomly 
    broken."""
    index_list = 


In [9]:
def neighbors(data, feature, impute_dict):
    """args: dataFrame with missing values, the feature to be imputed,
    and the dictionary in which to store the imputed values"""
    data = data[['Age','ID','Gender',feature]]
    miss_feat = data[data[feature].isna()]
    miss_feat_m = miss_feat[miss_feat.Gender == 'Male']
    miss_feat_f = miss_feat[miss_feat.Gender == 'Female']
    has_feat_mask = ~data.index.isin(miss_feat.index)
    others_m = data[np.bitwise_and(has_feat_mask,data.Gender=='Male')]
    others_f = data[np.bitwise_and(has_feat_mask,data.Gender=='Female')]
    for sex in ['Male','Female']:
        if sex == 'Female':
            for inx in miss_feat_f.index:   
                age_diff = others_f.Age - miss_feat_f['Age'][inx]
                
                nearest_inx = age_diff.abs().sort_values()[0:5].index
                nearest_mean = others_f[feature][nearest_inx].values.mean()
                if inx in impute_dict:
                    impute_dict[inx] = impute_dict[inx].update({feature:
                                                    (nearest_mean,nearest_inx)})
                    
                else:
                    impute_dict[inx] = {feature: (nearest_mean,nearest_inx)}
        else:
            for inx in miss_feat_m.index:
                age_diff = others_m.Age - miss_feat_m['Age'][inx]
                nearest_inx = age_diff.abs().sort_values()[0:5].index
                nearest_mean = others_m[feature][nearest_inx].values.mean()
                if inx in impute_dict:
                    impute_dict[inx].update({feature:
                                                    (nearest_mean,nearest_inx)})
                else:
                    impute_dict[inx] = {feature: (nearest_mean,nearest_inx)}
    return impute_dict

In [10]:
full_impute_dict = {}
for feature in missing_features:
    full_impute_dict.update(neighbors(sleep,feature,full_impute_dict))

In [11]:
full_impute_dict

{19: {'Awakenings': (1.0, Index([222, 67, 366, 199, 41], dtype='int64'))},
 85: {'Awakenings': (1.4, Index([233, 340, 7, 97, 344], dtype='int64'))},
 123: {'Awakenings': (2.0, Index([424, 54, 31, 451, 65], dtype='int64'))},
 138: {'Awakenings': (1.4, Index([420, 72, 11, 438, 80], dtype='int64'))},
 143: {'Awakenings': (2.0, Index([125, 345, 321, 439, 313], dtype='int64'))},
 170: {'Awakenings': (2.0, Index([30, 444, 192, 266, 20], dtype='int64'))},
 249: {'Awakenings': (1.2, Index([379, 210, 209, 200, 155], dtype='int64'))},
 337: {'Awakenings': (1.0, Index([222, 67, 366, 199, 41], dtype='int64'))},
 352: {'Awakenings': (1.0, Index([222, 67, 366, 199, 41], dtype='int64'))},
 404: {'Awakenings': (2.0, Index([30, 444, 192, 266, 20], dtype='int64')),
  'Caffeine consumption': (10.0,
   Index([444, 192, 107, 266, 30], dtype='int64'))},
 135: {'Awakenings': (0.4, Index([264, 184, 277, 174, 387], dtype='int64'))},
 149: {'Awakenings': (2.4, Index([128, 171, 310, 44, 98], dtype='int64'))},
 2

Now take the mean values and add to the sleepframes

In [12]:
for inx in full_impute_dict:
    for feat in full_impute_dict[inx]:
        sleep.loc[inx,feat] = full_impute_dict[inx][feat][0]

##### Q5 
divide all people into the following groups:
* Group 1: children (1-12)
* Group 2: teenagers (13-17)
* Group 3: young adults (18-30)
* Group 4: adults (31-60)
* Group 5: older adults (65+)

For each group, compute the mean and standard deviation
and summarize them in three tables (like one below): one
table for females, one tables for males and one combined

<img src='./table_q5.png' width=400 height=300/>


In [13]:
sleep.columns

Index(['ID', 'Age', 'Gender', 'Bedtime', 'Wakeup time', 'Sleep duration',
       'Sleep efficiency', 'REM sleep percentage', 'Deep sleep percentage',
       'Light sleep percentage', 'Awakenings', 'Caffeine consumption',
       'Alcohol consumption', 'Smoking status', 'Exercise frequency'],
      dtype='object')

In [14]:
sleep['Smoking status'] = [1 if 'Yes' else 0 for smoke in sleep['Smoking status']]

In [15]:
sleep_numeric = sleep.select_dtypes(include=['int64','float64']).copy()

In [16]:
metric_cols = ['Age','Sleep duration','Sleep efficiency', 'REM sleep percentage', 'Deep sleep percentage',
       'Light sleep percentage', 'Awakenings','Smoking status','Exercise frequency']

In [26]:
age_groups = [('children',1,12),('teenagers',13,17),('young_adults',18,30),
          ('adults',31,60),('older_adults',65,125)]
females = sleep[sleep.Gender=='Female']
males = sleep[sleep.Gender=='Male']

In [28]:
females[females.Age<13].describe()

,ID,Age,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
count,4.000000,4.000000,4.000000,4.000000,4.0,4.0,4.0,4.000000,4.0,4.000000,4.0,4.0
mean,152.000000,10.500000,8.375000,0.542500,18.0,35.0,45.0,2.500000,0.0,2.750000,1.0,0.0
std,148.689834,1.290994,0.946485,0.017078,0.0,0.0,0.0,1.290994,0.0,1.707825,0.0,0.0
min,10.000000,9.000000,7.000000,0.520000,18.0,35.0,45.0,1.000000,0.0,1.000000,1.0,0.0
25%,42.250000,9.750000,8.125000,0.535000,18.0,35.0,45.0,1.750000,0.0,1.750000,1.0,0.0
50%,132.500000,10.500000,8.750000,0.545000,18.0,35.0,45.0,2.500000,0.0,2.500000,1.0,0.0
75%,242.250000,11.250000,9.000000,0.552500,18.0,35.0,45.0,3.250000,0.0,3.500000,1.0,0.0
max,333.000000,12.000000,9.000000,0.560000,18.0,35.0,45.0,4.000000,0.0,5.000000,1.0,0.0


In [62]:
def data_msg(m,s):
    """to handle lack of data to compute mean, std"""
    try: msg = "mean: {:.2f}, std: {:.2f}".format(m,s)
    except ValueError: msg = 'No_Data'
    return msg

In [63]:
def summary_stats(data, age_group):
    std_dev = data[(data.Age>=age_group[1]) & 
                   (data.Age<=age_group[2])][metric_cols].std()
    mean = data[(data.Age>=age_group[1]) & 
                   (data.Age<=age_group[2])][metric_cols].mean()
    std_dev = std_dev.fillna(value='No_Data')
    mean = mean.fillna(value='No_Data')
    metrics = zip(metric_cols,[data_msg(m,s) for m,s 
                               in zip (mean,std_dev)])
    return {v[0]:v[1] for v in metrics}

In [43]:
age_group = age_groups[0]

std_dev = males[(males.Age>=age_group[1]) & 
                   (males.Age<=age_group[2])][metric_cols].std()
mean = males[(males.Age>=age_group[1]) & 
               (males.Age<=age_group[2])][metric_cols].mean()

In [44]:
metrics = zip(metric_cols,["mean: {:.2f}, std: {:.2f}".format(m,s) for m,s 
                               in zip (mean,std_dev)])

In [72]:
male_ag_df = pd.DataFrame()
for ag in age_groups:
    male_ag_df[ag[0]] = pd.Series(summary_stats(males, ag))
male_ag_df.index.name = 'Metric'
male_ag_df.columns.name = 'Age_Group'

In [73]:
male_ag_df

Age_Group,children,teenagers,young_adults,adults,older_adults
Metric,,,,,
Age,No_Data,No_Data,"mean: 24.47, std: 2.99","mean: 48.65, std: 6.95","mean: 66.11, std: 1.54"
Sleep duration,No_Data,No_Data,"mean: 7.45, std: 0.66","mean: 7.40, std: 0.91","mean: 7.44, std: 1.04"
Sleep efficiency,No_Data,No_Data,"mean: 0.75, std: 0.14","mean: 0.81, std: 0.12","mean: 0.74, std: 0.11"
REM sleep percentage,No_Data,No_Data,"mean: 22.05, std: 4.54","mean: 22.39, std: 3.79","mean: 23.78, std: 3.80"
Deep sleep percentage,No_Data,No_Data,"mean: 50.90, std: 19.08","mean: 55.13, std: 13.89","mean: 50.44, std: 16.43"
Light sleep percentage,No_Data,No_Data,"mean: 28.12, std: 17.13","mean: 22.58, std: 13.23","mean: 27.44, std: 17.26"
Awakenings,No_Data,No_Data,"mean: 1.95, std: 1.37","mean: 1.62, std: 1.32","mean: 2.67, std: 1.41"
Smoking status,No_Data,No_Data,"mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00"
Exercise frequency,No_Data,No_Data,"mean: 2.14, std: 1.06","mean: 2.12, std: 1.27","mean: 2.56, std: 0.88"


In [74]:
female_ag_df = pd.DataFrame()
for ag in age_groups:
    female_ag_df[ag[0]] = pd.Series(summary_stats(females, ag))
female_ag_df.index.name = 'Metric'
female_ag_df.columns.name = 'Age_Group'

In [75]:
female_ag_df

Age_Group,children,teenagers,young_adults,adults,older_adults
Metric,,,,,
Age,"mean: 10.50, std: 1.29","mean: 15.00, std: 1.58","mean: 26.38, std: 3.26","mean: 41.20, std: 7.39","mean: 65.00, std: 0.00"
Sleep duration,"mean: 8.38, std: 0.95","mean: 7.70, std: 0.67","mean: 7.56, std: 0.87","mean: 7.44, std: 0.87","mean: 7.42, std: 1.02"
Sleep efficiency,"mean: 0.54, std: 0.02","mean: 0.63, std: 0.04","mean: 0.79, std: 0.14","mean: 0.81, std: 0.14","mean: 0.79, std: 0.17"
REM sleep percentage,"mean: 18.00, std: 0.00","mean: 18.80, std: 1.79","mean: 25.07, std: 3.28","mean: 23.39, std: 3.92","mean: 22.00, std: 3.58"
Deep sleep percentage,"mean: 35.00, std: 0.00","mean: 32.60, std: 5.37","mean: 53.68, std: 14.55","mean: 53.11, std: 15.19","mean: 50.83, std: 19.49"
Light sleep percentage,"mean: 45.00, std: 0.00","mean: 46.80, std: 4.02","mean: 24.13, std: 14.32","mean: 24.08, std: 15.28","mean: 27.00, std: 18.99"
Awakenings,"mean: 2.50, std: 1.29","mean: 2.00, std: 1.00","mean: 1.44, std: 1.33","mean: 1.51, std: 1.31","mean: 1.33, std: 1.63"
Smoking status,"mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00"
Exercise frequency,"mean: 0.00, std: 0.00","mean: 0.00, std: 0.00","mean: 1.47, std: 1.33","mean: 1.69, std: 1.65","mean: 0.50, std: 1.22"


In [77]:
all_genders = pd.DataFrame()
for ag in age_groups:
    all_genders[ag[0]] = pd.Series(summary_stats(females, ag))
all_genders.index.name = 'Metric'
all_genders.columns.name = 'Age_Group'
all_genders

Age_Group,children,teenagers,young_adults,adults,older_adults
Metric,,,,,
Age,"mean: 10.50, std: 1.29","mean: 15.00, std: 1.58","mean: 26.38, std: 3.26","mean: 41.20, std: 7.39","mean: 65.00, std: 0.00"
Sleep duration,"mean: 8.38, std: 0.95","mean: 7.70, std: 0.67","mean: 7.56, std: 0.87","mean: 7.44, std: 0.87","mean: 7.42, std: 1.02"
Sleep efficiency,"mean: 0.54, std: 0.02","mean: 0.63, std: 0.04","mean: 0.79, std: 0.14","mean: 0.81, std: 0.14","mean: 0.79, std: 0.17"
REM sleep percentage,"mean: 18.00, std: 0.00","mean: 18.80, std: 1.79","mean: 25.07, std: 3.28","mean: 23.39, std: 3.92","mean: 22.00, std: 3.58"
Deep sleep percentage,"mean: 35.00, std: 0.00","mean: 32.60, std: 5.37","mean: 53.68, std: 14.55","mean: 53.11, std: 15.19","mean: 50.83, std: 19.49"
Light sleep percentage,"mean: 45.00, std: 0.00","mean: 46.80, std: 4.02","mean: 24.13, std: 14.32","mean: 24.08, std: 15.28","mean: 27.00, std: 18.99"
Awakenings,"mean: 2.50, std: 1.29","mean: 2.00, std: 1.00","mean: 1.44, std: 1.33","mean: 1.51, std: 1.31","mean: 1.33, std: 1.63"
Smoking status,"mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00","mean: 1.00, std: 0.00"
Exercise frequency,"mean: 0.00, std: 0.00","mean: 0.00, std: 0.00","mean: 1.47, std: 1.33","mean: 1.69, std: 1.65","mean: 0.50, std: 1.22"
